# Analytical dashboard

> This dashboard helps you explore the penguins dataset

Made with ❤️ by **[Ploomber cloud](https://platform.ploomber.io/)**.

## Any feedback?
We'd love to hear your feedback, and you'll get free compute, credits and Ploomber swag.

Set a [short meeting](https://calendly.com/ido-m/engineering), or [fill out this form](https://ploomber.io/contact/) to learn more.

In [3]:
from urllib.request import urlretrieve

import ipywidgets as widgets
import matplotlib.pyplot as plt
from jinja2 import Template

from sql.ggplot import ggplot, aes, geom_boxplot, geom_histogram, facet_wrap

In [4]:
_ = urlretrieve("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv", "penguins.csv")

In [5]:
%load_ext sql
%sql duckdb://
%config SqlMagic.displaycon = False

Found pyproject.toml from '/Users/idomi/Documents/ploomber/cloud-backend'

In [6]:
%%sql
CREATE TABLE penguins AS
SELECT *
FROM 'penguins.csv'
WHERE SEX IS NOT NULL

Count


In [7]:
islands = %sql select DISTINCT(island) from penguins
islands = [island[0] for island in islands]

species = %sql select DISTINCT(species) from penguins
species = [s[0] for s in species]

sex = %sql select DISTINCT(sex) from penguins
sex = [s[0] for s in sex]

In [8]:
from IPython import get_ipython
ip = get_ipython()
sql_magic = ip.find_cell_magic("sql")

## Data selection

> Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks or arrow keys.

In [9]:
widget_island = widgets.SelectMultiple(
    options=islands,
    description='Island',
    value=islands,
)

widget_species = widgets.SelectMultiple(
    options=species,
    description='Species',
    value=species,
)


widget_sex = widgets.SelectMultiple(
    options=sex,
    description='Sex',
    value=sex,
)

button = widgets.Button(
    description='Click me',
    tooltip='Plot',
)


widgets.TwoByTwoLayout(top_left=widget_island,
               bottom_left=widget_species,
               bottom_right=widget_sex)

TwoByTwoLayout(children=(SelectMultiple(description='Island', index=(0, 1, 2), layout=Layout(grid_area='top-le…

In [10]:
output1 = widgets.Output()
output2 = widgets.Output()
output3 = widgets.Output()
output4 = widgets.Output()

In [11]:
def compile_query(islands, species, sex):
    islands_in = ", ".join(repr(i) for i in islands)
    species_in = ", ".join(repr(i) for i in species)
    sex_in = ", ".join(repr(i) for i in sex)
    return Template("""
    SELECT * FROM penguins
    WHERE ISLAND IN ({{islands_in}})
    AND SPECIES IN ({{species_in}})
    AND SEX IN ({{sex_in}})
    """).render(islands_in=islands_in,
               species_in=species_in,
               sex_in=sex_in)

In [12]:
def plot_factory(column, fill):
    def plot():
        p = (ggplot("plotdata", with_="plotdata", mapping=aes(x=column))
                 + geom_histogram(bins=20, fill=fill))
        p.figure.axes[0].set_title(f"{column} by {fill}")
        plt.show(p.figure)
        return p
    
    return plot
    
mass_by_island = plot_factory("body_mass_g", "island")
mass_length_mm_by_species = plot_factory("body_mass_g", "species")
mass_length_mm_by_sex = plot_factory("body_mass_g", "sex")

In [13]:
def button_clicked(button):
    output1.clear_output()
    output2.clear_output()
    output3.clear_output()
    output4.clear_output()
    
    q = compile_query(widget_island.value,
                      widget_species.value,
                      widget_sex.value)
    
    ip.run_cell_magic("sql", "--save plotdata", q)

    with output1:
        mass_by_island()

    with output2:
        mass_length_mm_by_species()

    with output3:
        mass_length_mm_by_sex()

    with output4:
        from IPython.display import display
        display(ip.run_cell_magic("sql", "", "SELECT * FROM plotdata"))



button = widgets.Button(
    description='Plot',
    tooltip='Plot the data',
)    

button.on_click(button_clicked)
button

Button(description='Plot', style=ButtonStyle(), tooltip='Plot the data')

## Plots

In [14]:
widgets.TwoByTwoLayout(top_left=output1,
                       top_right=output2,
                       bottom_left=output3)

TwoByTwoLayout(children=(Output(layout=Layout(grid_area='top-left')), Output(layout=Layout(grid_area='top-righ…

## Data preview

In [15]:
output4

Output()